# Shor's Algorithm

Shor's algorithm accepts a (large) integer $N$ and finds $p$, $q$ such that $N=p\cdot q$ and both $p$, $q$ are prime

1. Choose a random integer $1 < a < N$
2. Compute $b = \gcd(a,N)$
3. If $b \neq 1$: $b$ factors $N$, return $p=b$ and $q=N/b$
4. Else:
5. Use quantum subroutine to find the order $r$ of $a$ (i.e., $\min(r>0)$ such that $a^r = 1\mod{N}$
6. If $r$ is odd: move back to line 1
7. Compute $g = \gcd(a^{r/2}\pm 1, N)$
8. If $g > 1$: return $p=g$, $q=N/g$
9. Else: go back to line 1

# Quantum Order-finding Algorithm

https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/algorithms/shor_algorithm.ipynb

This algorithm will take two coprime integers, $a$ and $N$, and output $r$ which is the period (i.e., $a^r\bmod{N}$)

**Input:**
- Two coprime integers $a$, $N$
- The operator $U$ such that $U\ket{x}\ket{y} = \ket{x}\ket{a^x\bmod N}$
- A state to store the function results, initialized to $\ket{0}$
- $n = O(L + \log{1/\epsilon})$ qubits initialized to $\ket{0}$ ($L$ is the bit length of $N$)

**Output:** The period $r$ of $a^x\bmod N$

1. Create the initial state $\ket{0}\ket{0}$
2. Apply a Hadamard gate to the first register to create the superposition $\frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1}\ket{x}\ket{0}$
3. Apply the operator $U$ to get
$$
\frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1}\ket{x}\ket{a^x\bmod N} \approx \frac{1}{\sqrt{r2^n}} \sum_{x=0}^{2^n-1} \sum_{l=0}^{r-1} e^{2\pi ilx/r}\ket{x}\ket{u_l}
$$
    where
$$
\ket{u_l} = \frac{1}{\sqrt{r}} \sum_{x=0}^{r-1}e^{-2\pi ilx/r}\ket{a^x\bmod N} \iff \ket{a^x\bmod N} = \frac{1}{\sqrt{r}} \sum_{l=0}^{r-1} e^{2\pi ilx/r}\ket{u_l}
$$

4. Apply the inverse Fourier transform on the first register to get
$$
\frac{1}{2^n\sqrt{r}} \sum_{l=0}^{r-1} \sum_{x=0}^{2^n-1} \sum_{k=0}^{2^n-1} e^{2\pi ilx/r} e^{-2\pi ixk/2^n} \ket{k} \ket{u_l} = \frac{1}{2^n\sqrt{r}} \sum_{l=0}^{r-1} \sum_{x=0}^{2^n-1} \sum_{k=0}^{2^n-1} e^{2\pi ix(l/r - k/2^n)} \ket{k} \ket{u_l}
$$

5. Measure the first register to get $k_0=2^n\frac{l}{r}$. The state will collapse to
$$
\frac{1}{2^n\sqrt{r}} \sum_{l=0}^{r-1} \sum_{x=0}^{2^n-1} e^{2\pi ix(l/r - k_0/2^n)} \ket{u_l} \equiv \frac{1}{\sqrt{r}} \sum_{l=0}^{r-1} \alpha_l \ket{u_l}
$$

    with probability

$$
\mathcal{P}_{k_0} = \frac{1}{r} \sum_{l=0}^{r-1} \left| \alpha_l \right|^2
$$

6. Apply the continued fractions algorithm to determine $r$.

### Imports

In [3]:
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister

In [19]:
n = 5
a, N = 13, 37

# initialize two registers of size n
ar = QuantumRegister(n, name='Argument Register')
fr = QuantumRegister(n, name='Function Register')
nr = QuantumRegister(n, name='Counting Register')
cr = ClassicalRegister(n, name='Classical Register')

# initialize circuit with registers
circuit = QuantumCircuit(nr, fr, ar, cr)

In [20]:
# apply Hadamard gate to first (argument) register
circuit.h(nr)

In [ ]:
# apply the operator U

# property: (a^n * x)mod N = (a..(a(ax)mod N)mod N).. )mod N) ?